### 1. Importar les llibreries

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import hashlib
import pickle
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, make_scorer

### 2. Carregar el dataset

In [3]:
df = pd.read_csv('/Users/nicolakorff/Desktop/ML/IT_Academy/bank_dataset_wins.CSV')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59.0,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56.0,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41.0,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55.0,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54.0,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


### 3. Divisió del dataset

In [23]:
# comprovar el balanç de classes
print(df['deposit'].value_counts(normalize=True) * 100)
print(df['job'].value_counts(normalize=True) * 100)
print(df['marital'].value_counts(normalize=True) * 100)
print(df['education'].value_counts(normalize=True) * 100)
print(df['default'].value_counts(normalize=True) * 100)
print(df['balance'].value_counts(normalize=True) * 100)
print(df['housing'].value_counts(normalize=True) * 100)
print(df['loan'].value_counts(normalize=True) * 100)
print(df['contact'].value_counts(normalize=True) * 100)
print(df['day'].value_counts(normalize=True) * 100)
print(df['month'].value_counts(normalize=True) * 100)
print(df['duration'].value_counts(normalize=True) * 100)
print(df['campaign'].value_counts(normalize=True) * 100)
print(df['pdays'].value_counts(normalize=True) * 100)
print(df['previous'].value_counts(normalize=True) * 100)
print(df['poutcome'].value_counts(normalize=True) * 100)

deposit
no     52.616019
yes    47.383981
Name: proportion, dtype: float64
job
management       22.988712
blue-collar      17.416234
technician       16.332199
admin.           11.951263
services          8.269127
retired           6.970077
self-employed     3.628382
student           3.225228
unemployed        3.198352
entrepreneur      2.938541
housemaid         2.454757
unknown           0.627128
Name: proportion, dtype: float64
marital
married     56.905978
single      31.522811
divorced    11.571211
Name: proportion, dtype: float64
education
secondary    49.072165
tertiary     33.034514
primary      13.446885
unknown       4.446437
Name: proportion, dtype: float64
default
no     98.494893
yes     1.505107
Name: proportion, dtype: float64
balance
 0       6.934241
 1       0.349400
 3       0.313564
 2       0.304605
 4       0.259810
           ...   
 5517    0.008959
 8332    0.008959
 4859    0.008959
-267     0.008959
-134     0.008959
Name: proportion, Length: 3805, dtype: fl

In [24]:
# Després de comprovar el balanç de les classes es divideix el dataset en conjunts d'entrenament i prova en un 80/20.
# S'ha considerat el 80/20 el més adient per què en general sol ser la més adient.
# concretament aqui també per la quatitat de dades, el balaç de les classes i tipus difernet de dades.
train, test = train_test_split(df, test_size=0.2, random_state=22)
print("Grandària del conjunt d'entrenament (train):", train.shape)
print("Grandària del conjunt de prova (test):", test.shape)

Grandària del conjunt d'entrenament (train): (8929, 17)
Grandària del conjunt de prova (test): (2233, 17)


In [5]:
# Desar els conjunts en fitxers CSV
train.to_csv('bank_dataset_train.csv', index=False)
test.to_csv('bank_dataset_test.csv', index=False)
# Comprovar les primeres files dels dos conjunts
train.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
843,44.0,technician,married,secondary,no,267,no,no,cellular,22,aug,520,2,-1,0,unknown,yes
1899,47.0,admin.,married,secondary,no,663,yes,yes,cellular,12,may,409,1,274,6,other,yes
7459,56.0,retired,married,tertiary,no,5769,no,no,unknown,20,jun,15,3,-1,0,unknown,no
2346,54.0,management,married,tertiary,no,1464,no,no,cellular,30,jun,157,1,-1,0,unknown,yes
5673,40.0,admin.,single,unknown,no,355,yes,no,cellular,7,apr,345,2,138,6,other,no


In [6]:
test.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
1904,37.0,admin.,single,primary,no,912,yes,no,cellular,12,may,637,3,328,22,other,yes
3113,52.0,services,married,secondary,no,659,no,no,cellular,28,jan,390,4,93,1,success,yes
1281,71.0,retired,married,tertiary,no,653,no,no,telephone,26,feb,367,1,-1,0,unknown,yes
7856,29.0,blue-collar,married,primary,no,59,yes,yes,cellular,29,jul,665,4,-1,0,unknown,no
6757,28.0,blue-collar,married,primary,no,278,yes,yes,telephone,15,may,31,9,373,4,other,no


- S' ajusta inicialment només el bloc de train, perquè el bloc de test romangui desconegut i evitar així el "data leakage".

### 4. Transformar variables categóriques

In [7]:
# Label Encoding per 'education', 'month', 'poutcome' al ser valors ordinaris 
# Veure els valors únics a la columna 'month'
unique_month = train['month'].unique()
print(unique_month)

['aug' 'may' 'jun' 'apr' 'jul' 'nov' 'jan' 'feb' 'mar' 'oct' 'sep' 'dec']


In [8]:
# Us de OrdinalEncoder
train2=train
# Ordre específic
month_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
# Configurar l'OrdinalEncoder amb l'ordre desitjat
ordinal_encoder = OrdinalEncoder(categories=[month_order])
# Ajustar i transformar la columna
train2['month_encoded'] = ordinal_encoder.fit_transform(train[['month']])

# Verificació
train2.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit,month_encoded
843,44.0,technician,married,secondary,no,267,no,no,cellular,22,aug,520,2,-1,0,unknown,yes,7.0
1899,47.0,admin.,married,secondary,no,663,yes,yes,cellular,12,may,409,1,274,6,other,yes,4.0
7459,56.0,retired,married,tertiary,no,5769,no,no,unknown,20,jun,15,3,-1,0,unknown,no,5.0
2346,54.0,management,married,tertiary,no,1464,no,no,cellular,30,jun,157,1,-1,0,unknown,yes,5.0
5673,40.0,admin.,single,unknown,no,355,yes,no,cellular,7,apr,345,2,138,6,other,no,3.0


In [9]:
# Veure els valors únics a la columna 'education'
unique_education = train2['education'].unique()
print(unique_education)

['secondary' 'tertiary' 'unknown' 'primary' nan]


In [10]:
# Us del mappeig manual, per endreçar ordinalment escullint l'enumaració
# Especificar l'ordre amb el diccionari
education_map = {'primary': 1, 'secondary': 2, 'tertiary': 3, 'unknown': 0}

# Aplicar el mappeig a la columna
train2['education_encoded'] = train2['education'].map(education_map)
# Verificació
train2.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit,month_encoded,education_encoded
843,44.0,technician,married,secondary,no,267,no,no,cellular,22,aug,520,2,-1,0,unknown,yes,7.0,2.0
1899,47.0,admin.,married,secondary,no,663,yes,yes,cellular,12,may,409,1,274,6,other,yes,4.0,2.0
7459,56.0,retired,married,tertiary,no,5769,no,no,unknown,20,jun,15,3,-1,0,unknown,no,5.0,3.0
2346,54.0,management,married,tertiary,no,1464,no,no,cellular,30,jun,157,1,-1,0,unknown,yes,5.0,3.0
5673,40.0,admin.,single,unknown,no,355,yes,no,cellular,7,apr,345,2,138,6,other,no,3.0,0.0


In [25]:
# Us del mappeig manual
# Especificar l'ordre amb el diccionari
poutcome_map = {'success': 3, 'failure': 2, 'other': 1, 'unknown': 0}

# Aplicar el mappeig a la columna
train2['poutcome_encoded'] = train2['poutcome'].map(poutcome_map)
 
# Verificació
train2.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit,month_encoded,education_encoded,poutcome_encoded
843,44.0,technician,married,secondary,no,267,no,no,cellular,22,aug,520,2,-1,0,unknown,yes,7.0,2.0,0
1899,47.0,admin.,married,secondary,no,663,yes,yes,cellular,12,may,409,1,274,6,other,yes,4.0,2.0,1
7459,56.0,retired,married,tertiary,no,5769,no,no,unknown,20,jun,15,3,-1,0,unknown,no,5.0,3.0,0
2346,54.0,management,married,tertiary,no,1464,no,no,cellular,30,jun,157,1,-1,0,unknown,yes,5.0,3.0,0
5673,40.0,admin.,single,unknown,no,355,yes,no,cellular,7,apr,345,2,138,6,other,no,3.0,0.0,1


In [ ]:
# Veure els valors únics a la columna 'poutcome'
unique_poutcome = train2['poutcome'].unique()
print(unique_poutcome)

In [56]:
# One-Hot Encoding per 'loan', 'housing', 'deafult', 'deposit', 'marital', 'contact' 'job', ja que no es poden ordenar de foma ordinal
train3 = train2

In [57]:
# Utilitzar get_dummies per a 'loan', 'housing', 'deafult', 'deposit', al ser columnes amb valors binaris
train3 = pd.get_dummies(train3, columns=['loan'], prefix='loan')
train3 = pd.get_dummies(train3, columns=['housing'], prefix='housing')
train3 = pd.get_dummies(train3, columns=['default'], prefix='default')
train3 = pd.get_dummies(train3, columns=['deposit'], prefix='deposit')

# Verificació
train3.head()

,age,job,marital,education,balance,contact,day,month,duration,campaign,...,education_encoded,poutcome_encoded,loan_no,loan_yes,housing_no,housing_yes,default_no,default_yes,deposit_no,deposit_yes
843,44.0,technician,married,secondary,267,cellular,22,aug,520,2,...,2.0,0,True,False,True,False,True,False,False,True
1899,47.0,admin.,married,secondary,663,cellular,12,may,409,1,...,2.0,1,False,True,False,True,True,False,False,True
7459,56.0,retired,married,tertiary,5769,unknown,20,jun,15,3,...,3.0,0,True,False,True,False,True,False,True,False
2346,54.0,management,married,tertiary,1464,cellular,30,jun,157,1,...,3.0,0,True,False,True,False,True,False,False,True
5673,40.0,admin.,single,unknown,355,cellular,7,apr,345,2,...,0.0,1,True,False,False,True,True,False,True,False


In [58]:
train4=train3

# Inicialitzar el OneHotEncoder
encoder = OneHotEncoder(drop='first') # 'drop="first"' elimina una categoria per evitar multicol·linealitat

# Aplicar l'encoder a la columna 'marital' i convertir en un array
marital_encoded = encoder.fit_transform(train4[['marital']]).toarray()

# Crear un DataFrame amb les columnes codificades i unir-les al DataFrame original
marital_encoded_train4 = pd.DataFrame(marital_encoded, columns=encoder.get_feature_names_out(['marital']))
train4 = pd.concat([train4, marital_encoded_train4], axis=1)

In [68]:
# Inicialitzar el OneHotEncoder
encoder = OneHotEncoder(drop='first') # 'drop="first"' elimina una categoria per evitar multicol·linealitat

# Aplicar l'encoder a la columna 'marital' i convertir en un array
contact_encoded = encoder.fit_transform(train4[['contact']]).toarray()

# Crear un DataFrame amb les columnes codificades i unir-les al DataFrame original
contact_encoded_train4 = pd.DataFrame(contact_encoded, columns=encoder.get_feature_names_out(['contact']))
train4 = pd.concat([train4, contact_encoded_train4], axis=1)

In [59]:
# Inicialitzar el OneHotEncoder
encoder = OneHotEncoder(drop='first') # 'drop="first"' elimina una categoria per evitar multicol·linealitat

# Aplicar l'encoder a la columna 'job' i convertir en un array
job_encoded = encoder.fit_transform(train4[['job']]).toarray()

# Crear un DataFrame amb les columnes codificades i unir-les al DataFrame original
job_encoded_train4 = pd.DataFrame(job_encoded, columns=encoder.get_feature_names_out(['job']))
train4 = pd.concat([train4, job_encoded_train4], axis=1)

In [69]:
train4.head()

,age,balance,contact,day,duration,campaign,pdays,previous,month_encoded,education_encoded,...,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,job_nan,contact_telephone,contact_unknown,contact_nan
843,44.0,267.0,cellular,22.0,520.0,2.0,-1.0,0.0,7.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1899,47.0,663.0,cellular,12.0,409.0,1.0,274.0,6.0,4.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7459,56.0,5769.0,unknown,20.0,15.0,3.0,-1.0,0.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2346,54.0,1464.0,cellular,30.0,157.0,1.0,-1.0,0.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5673,40.0,355.0,cellular,7.0,345.0,2.0,138.0,6.0,3.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
# Eliminar columnes originals:
train4.drop("month", axis="columns", inplace=True)
train4.drop("education", axis="columns", inplace=True)
train4.drop("poutcome", axis="columns", inplace=True) 
train4.drop("job", axis="columns", inplace=True)
train4.drop("marital", axis="columns", inplace=True)
train4.drop("contact", axis="columns", inplace=True)

In [75]:
# Verificació
train4.head()

,age,balance,day,duration,campaign,pdays,previous,month_encoded,education_encoded,poutcome_encoded,...,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,job_nan,contact_telephone,contact_unknown,contact_nan
843,44.0,267.0,22.0,520.0,2.0,-1.0,0.0,7.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1899,47.0,663.0,12.0,409.0,1.0,274.0,6.0,4.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7459,56.0,5769.0,20.0,15.0,3.0,-1.0,0.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2346,54.0,1464.0,30.0,157.0,1.0,-1.0,0.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5673,40.0,355.0,7.0,345.0,2.0,138.0,6.0,3.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### 5. Estandarització o Normalització de les dades

In [81]:
# Convé estandaritzar les variables numèriques que tenen una escala àmplia i que podrien afectar els models
# Especificar les columnes numèriques a estandarditzar
columns_to_standardize = ['age', 'balance', 'duration', 'campaign', 'pdays', 'previous']
# Inicialitzar l'escalador
scaler = StandardScaler()
# Aplicar l'estandardització només a les columnes numèriques
train4[columns_to_standardize] = scaler.fit_transform(train4[columns_to_standardize])
# Desar el DataFrame estandarditzat en un fitxer CSV
train4.to_csv('bank_dataset_train.csv', index=False)
# Verificar
train4.describe()

,age,balance,day,duration,campaign,pdays,previous,month_encoded,education_encoded,poutcome_encoded,...,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,job_nan,contact_telephone,contact_unknown,contact_nan
count,8.921000e+03,8.929000e+03,8929.000000,8.929000e+03,8.929000e+03,8.929000e+03,8.929000e+03,8929.000000,8922.000000,8929.000000,...,10737.000000,10737.000000,10737.000000,10737.000000,10737.000000,10737.000000,10737.000000,11086.000000,11086.000000,11086.000000
mean,-7.964833e-18,6.366157e-18,15.579572,4.177791e-18,-3.262656e-17,-6.764042e-18,-2.466886e-17,5.202150,2.107039,0.547990,...,0.030362,0.068734,0.026823,0.134488,0.026451,0.005309,0.168390,0.054664,0.169583,0.194570
std,1.000056e+00,1.000056e+00,8.390087,1.000056e+00,1.000056e+00,1.000056e+00,1.000056e+00,2.554998,0.791076,1.017416,...,0.171590,0.253014,0.161574,0.341192,0.160479,0.072671,0.374229,0.227333,0.375283,0.395887
min,-1.946549e+00,-2.538370e+00,1.000000,-1.065947e+00,-5.465483e-01,-4.759835e-01,-3.594093e-01,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-7.703774e-01,-4.207304e-01,8.000000,-6.727830e-01,-5.465483e-01,-4.759835e-01,-3.594093e-01,4.000000,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-1.822918e-01,-2.927111e-01,15.000000,-3.374374e-01,-1.881369e-01,-4.759835e-01,-3.594093e-01,5.000000,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6.578306e-01,4.968720e-02,21.000000,3.563810e-01,1.702745e-01,-4.759835e-01,-3.594093e-01,7.000000,3.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,4.522394e+00,2.423651e+01,31.000000,1.014789e+01,2.167496e+01,7.318944e+00,2.538137e+01,11.000000,3.000000,3.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
